In [170]:
from bs4 import BeautifulSoup
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import string
import math
import time
import os

In [171]:
tokenized_files_path = 'input/tokens'

stopwords_file_path = 'input/stopwords.txt'

In [172]:
start = time.time()

In [173]:
stopwords = set()

with open(stopwords_file_path, 'r') as f:
    for word in f:
        stopwords.add(word.strip())
print(time.time() - start)

0.005446910858154297


In [174]:
term_freq = defaultdict(int)
words_in_doc = defaultdict(list)
inverted_index = defaultdict(set)

tokenized_files = os.listdir(tokenized_files_path)

for file in tokenized_files:
    [doc_number, format] = file.split('.')
    doc_number = int(doc_number)
    file_path = 'input/tokens/' + file
    with open(file_path) as f:
        for word in f:
            word = word.strip()
            if word not in stopwords and word != '':
                term_freq[word] += 1
                words_in_doc[doc_number].append(word)
                inverted_index[word].add(doc_number)

In [175]:
term_freq = {i: term_freq[i] for i in term_freq.keys() if term_freq[i] > 1}

In [176]:
for key, value in words_in_doc.items():
    words_in_doc[key] = [i for i in value if i in term_freq.keys()]

print(time.time() - start)

0.7576990127563477


In [177]:
TF = defaultdict(dict)
IDF = {}

for doc_number, words in tqdm(words_in_doc.items()):
    for word in words:
        if word not in TF[doc_number]:
            TF[doc_number][word] = words.count(word) / len(words)
        if word not in IDF:
            IDF[word] = math.log(len(tokenized_files) / len(inverted_index[word]))

print(time.time() - start)

100%|█████████████████████████████████████████| 501/501 [00:11<00:00, 43.94it/s]

12.17963695526123


In [178]:
all_words = set().union(*words_in_doc.values())
matrix = np.zeros((len(tokenized_files), len(all_words)))

for doc_number, words in words_in_doc.items():
    words_set = set(words)
    for j, term in enumerate(all_words):
        if term in words_set:
            matrix[doc_number - 1, j] = TF[doc_number][term] * IDF[term]

print(time.time() - start)

14.025120973587036


In [179]:
# Compute cosine similarity
def cosine_similarity(x, y):
    prod = np.dot(x, y)
    x_val = np.sqrt(np.sum(x**2)) 
    y_val = np.sqrt(np.sum(y**2))
    cosine_similarity = prod / (x_val * y_val)
    
    return cosine_similarity

In [180]:
similarity_matrix = np.zeros((len(tokenized_files), len(tokenized_files)))

for i in range(len(tokenized_files)):
    for j in range(i+1, len(tokenized_files)):
        similarity_matrix[i][j] = cosine_similarity(matrix[i], matrix[j])
np.fill_diagonal(similarity_matrix, 1)

/var/folders/x3/lxxrgs757cq7rxgtscwjyclh0000gn/T/ipykernel_74298/974850055.py:6: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = prod / (x_val * y_val)


In [181]:
def get_similar_pair(matrix):
    matrix_copy = matrix.copy()
    np.fill_diagonal(matrix_copy, -np.inf)

    max_index_flat = np.nanargmax(matrix_copy)
    max_indices = np.unravel_index(max_index_flat, matrix.shape)

    max_similarity = matrix[max_indices]
    document_pair = tuple(f"{index + 1}.html" for index in max_indices)
    return document_pair, max_similarity

In [182]:
def get_dissimilar_pair(matrix):
    matrix_copy = matrix.copy()
    np.fill_diagonal(matrix_copy, np.inf)

    min_index_flat = np.nanargmin(matrix_copy)
    min_indices = np.unravel_index(min_index_flat, matrix.shape)

    min_similarity = matrix[min_indices]
    document_pair = tuple(f"{index + 1}.html" for index in min_indices)
    return document_pair, min_similarity

In [183]:
highest = get_most_similar_pair(similarity_matrix)
print(f"The most similar pair is: {highest[0]} with similarity score of {highest[1]:.5f}")

The most similar pair is: ('102.html', '130.html') with similarity score of 1.00000


In [184]:
lowest = get_most_dissimilar_pair(similarity_matrix)
print(f"The least similar pair is: {lowest[0]} with similarity score of", lowest[1])

The least similar pair is:('1.html', '13.html') with similarity score of 0.0


In [185]:
np.fill_diagonal(similarity_matrix, 1)
corpus_centroid = np.mean(similarity_matrix, axis=1)

# Find the index of the document with the highest average similarity value
closest_document_index = np.argmax(corpus_centroid)
print("Closest document to the centroid of the corpus:",  str(closest_document_index + 1) + '.html')

Closest document to the centroid of the corpus: 1.html


In [186]:
def agglomerative_clustering_custom(distance_matrix, method='complete', threshold=0.6):
    # Define functions for each linkage method
    def single_linkage(cluster1, cluster2):
        return np.min(distance_matrix[np.ix_(cluster1, cluster2)])

    def complete_linkage(cluster1, cluster2):
        return np.max(distance_matrix[np.ix_(cluster1, cluster2)])

    def centroid_linkage(cluster1, cluster2):
        centroid1 = np.mean(distance_matrix[cluster1, :], axis=0)
        centroid2 = np.mean(distance_matrix[cluster2, :], axis=0)
        return np.linalg.norm(centroid1 - centroid2)
    
    # Mapping of method names to functions
    methods = {
        'single': single_linkage,
        'complete': complete_linkage,
        'centroid': centroid_linkage
    }
    
    # Initialize clusters
    clusters = [[i] for i in range(len(distance_matrix))]

    # Initialize cluster merge function
    cluster_dist = methods[method]

    while True:
        # Find the two closest clusters
        cluster_to_merge = min(
            ((i, j) for i in range(len(clusters)) for j in range(i + 1, len(clusters))),
            key=lambda x: cluster_dist(clusters[x[0]], clusters[x[1]])
        )
        
        # Calculate distance between the two closest clusters
        min_distance = cluster_dist(clusters[cluster_to_merge[0]], clusters[cluster_to_merge[1]])

        # Stop if distance is above the threshold
        if min_distance > threshold:
            break

        # Merge the two closest clusters
        clusters[cluster_to_merge[0]].extend(clusters[cluster_to_merge[1]])
        clusters.pop(cluster_to_merge[1])
        
        merger = tuple(f"{num + 1}.html" for num in clusters[cluster_to_merge[0]])
        with open("clusters.txt", "a") as f:
            f.write(f"Merged clusters: {merger}\n")

    return clusters

In [187]:
print("Complete link method:")
complete_clusters = agglomerative_clustering_custom(similarity_matrix, method='complete')
print("Result:", complete_clusters)
end = time.time()

Complete link method:
Result: [[0, 12, 101, 152, 157, 175, 185, 487, 108, 186, 121, 290, 58, 103, 482, 1, 454, 41, 80, 164, 412, 474, 500, 84, 372, 113, 360, 275, 389, 387, 5, 6, 481, 25, 68, 133, 401, 464, 159, 354, 296, 10, 399, 455, 174, 162, 390, 273, 352, 338, 143, 254, 249, 235, 222, 180], [2, 7, 31, 34, 89, 126, 470, 495, 212, 326, 262, 260, 244, 214, 365, 43, 439, 480, 120, 194, 46, 477, 380, 111, 461, 502, 293, 243, 127, 137, 421, 136, 37, 213, 370, 300, 116, 171, 79, 98, 405, 496, 351, 94, 86], [3, 13, 99, 147, 154, 462, 445, 425, 391, 384, 167, 228, 14, 82, 437, 456, 42, 138, 176, 329, 369, 337, 288, 28, 410, 467, 285, 281, 266, 141, 237, 236, 204, 166, 184, 151], [4, 32, 129, 199, 255, 8, 26, 473, 30, 45, 458, 253, 117, 74, 179, 448, 356, 104, 78, 335, 19, 24, 438, 459, 308, 302, 297, 274, 145, 16, 69, 181, 449, 383, 301, 362, 18, 466, 40, 230, 400, 177, 494, 118, 63, 488, 378, 291, 172, 211, 271, 268, 426, 33, 71, 472, 342, 267, 339, 233, 140, 429, 146], [9, 22, 60, 398, 4

In [188]:
print("Total time taken: ", end - start)

Total time taken:  181.8699610233307
